In [35]:
import evaluate
import pandas as pd
import numpy as np

from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import (AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback)

#### Переменные

In [36]:
DATASET_NAME = "./datasets/ru-plus.csv"
TEST_SIZE = 0.2
MODEL_NAME = "ai-forever/sbert_large_nlu_ru"
SAVE_DIRECTORY = "./models/sbert_plus"
OUTPUT_LOG_NAME = "./output/sbert_plus"

#### Загружаем данные

In [37]:
df = pd.read_csv(DATASET_NAME, delimiter="|")
df.columns = ["text", "label"]
df['label'] = df['label'].astype(int)

#### Конвертируем датасет в Dataset

In [38]:
train, test_valid = train_test_split(df, test_size=TEST_SIZE, shuffle=True)
train = Dataset.from_pandas(train)
test, valid = train_test_split(test_valid, test_size=0.5)
test = Dataset.from_pandas(test)
valid = Dataset.from_pandas(valid)

#### Выполняем предобработку текста

In [39]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=64)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenized_train = train.map(tokenize_function)
tokenized_test = test.map(tokenize_function)
tokenized_valid = valid.map(tokenize_function)

Map:   0%|          | 0/185 [00:00<?, ? examples/s]

Map:   0%|          | 0/23 [00:00<?, ? examples/s]

Map:   0%|          | 0/24 [00:00<?, ? examples/s]

#### Загружаем предобученную модель

In [50]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai-forever/sbert_large_nlu_ru and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### Задаем параметры обучения

In [51]:
training_args = TrainingArguments(
    output_dir=OUTPUT_LOG_NAME,
    learning_rate=2e-5,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy='epoch',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=32,
    load_best_model_at_end=True,
    report_to="none"
)

#### Определяем как считать метрику

In [52]:
metric = evaluate.load("f1")

#### Выполняем обучение

In [53]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average='macro')

early_stopper = EarlyStoppingCallback(early_stopping_threshold=0.0001)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    compute_metrics=compute_metrics,
    callbacks=[early_stopper]

)
trainer.train()

C:\Users\Danilmet\PycharmProjects\tg_bot\venv\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1
1,0.842400,0.580079,0.766234
2,0.293600,0.170170,1.000000
3,0.077500,0.032700,1.000000
4,0.019000,0.009024,1.000000
5,0.006600,0.004910,1.000000
6,0.003900,0.003252,1.000000
7,0.002700,0.002518,1.000000
8,0.002300,0.002129,1.000000
9,0.002000,0.001870,1.000000
10,0.001800,0.001673,1.000000


TrainOutput(global_step=156, training_loss=0.09662464510601683, metrics={'train_runtime': 81.2655, 'train_samples_per_second': 72.848, 'train_steps_per_second': 4.725, 'total_flos': 280162812708480.0, 'train_loss': 0.09662464510601683, 'epoch': 13.0})

#### Сохраняем модель

In [54]:
tokenizer.save_pretrained(SAVE_DIRECTORY)
model.save_pretrained(SAVE_DIRECTORY)